# Credmark - Bollinger Bands Simulation
## Uniswap V3 Position recommentation

### Bollinger Bands - Uniswap v3 Position Recommendation

#### NOTE: This notebook is structured for a deployment format for AWS Lambda
#### Specific comments are mentioned for sections that need to be changed to switch between AWS Lambda and local testing

### Import Packages

In [1]:
# These packages have to be locally installed for lambda deployment using:
# pip install -t . <package-name>
# 
# For Local deployment use:
# pip install <package-name>
import pandas as pd
import numpy as np
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [2]:
# Built-in packages
import json
from datetime import datetime
import math

#### Initial Declarations

In [3]:
# Subgraph URL used for Uniswap V3
## This is the one of the official testing subgraphs, other subgraphs may be used as well, should work the same
# Link for testing subgraph: https://thegraph.com/explorer/subgraph/ianlapham/uniswap-v3-testing
GRAPH_URL_TESTING = "https://api.thegraph.com/subgraphs/name/ianlapham/uniswap-v3-testing"

In [4]:
# Graph API Call Dictionary
## Currently supports "poolDayDatas", others can be similarly added
## It's currently structured to allow for address and date of recommendation to be inserted via user input
GRAPH_API_DICT = {
    "poolDayDatas" : [
    ["(where: {pool: \"", "\" ,  date_lt: ", " }){ "],
     ["date", 
     "sqrtPrice", 
     "token0Price", 
     "token1Price", 
     "tick", 
     "tvlUSD", 
     "volumeToken0", 
     "volumeToken1", 
     "volumeUSD", 
     "txCount", 
     "}"]

     ]
     }

## Sample Query will look like:
'''
{
    poolDayDatas(where: {pool: "0x7858e59e0c01ea06df3af3d20ac7b0003275d4bf" ,  date_lt: 1625855400 }){
        date
        sqrtPrice
        token0Price
        token1Price
        tick
        tvlUSD
        volumeToken0
        volumeToken1
        volumeUSD
        txCount
    }
}
'''
## "0x7858e59e0c01ea06df3af3d20ac7b0003275d4bf" : address of the v3 pool
## "1625855400": date to limit the subgraph data to this date and get the prediction, in UNIX Timestamp format
## User input for date is "dd-mm-yyyy" format, a converter method will be used to achieve the above query
print()

In [5]:
# feeMapping: for feeAmount and tickSpacing mapping
feeMapping = {
    "500":10,
    "3000":60,
    "10000":200
}

### Helper Methods

In [6]:
# Convert date to timestamp format
def dateToEpoch(val):
    '''
        Inputs the date in dd-mm-yyyy format and converts to timestamp to be used in subgraph query
    '''
    d, m, y = val.split("-")
    ts= datetime(int(y), int(m),int(d), 0, 0).timestamp()
    return int(ts)

In [7]:
# Calculating current tick value from price
## NOTE: This is different from finding the tick of lower and upper range of the position
## Current price uses floor method for the tick value as mention in the Uniswap V3 Whitepaper
## Uniswap V3 Whitepaper Link: https://uniswap.org/whitepaper-v3.pdf
def tickcalc(input_data):
    return math.floor(np.log(math.sqrt(input_data)) / np.log(math.sqrt(1.0001)))

In [8]:
# Calculating tick from price, used for position ranges
def pricetickcalc(input_data):
    return np.log(math.sqrt(input_data))/np.log(math.sqrt(1.0001))

In [9]:
# Get price for a particular tick
def priceFromTick(input_data):
    return pow(1.0001, input_data)

In [10]:
# Formatting the date
def DateTimeConverter(input_date):
    return datetime.utcfromtimestamp(input_date).strftime('%Y-%m-%d %H:%M:%S')

#### AWS Style Lambda Handler

In [11]:
def lambda_handler(event, context):
    # Lambda receives the input in such format
    # Event's body parameter will be used to extract the input values
    '''
        Final Input Call:
        {
          "body": {
            "query": "poolDayDatas",
            "address": "0x7858e59e0c01ea06df3af3d20ac7b0003275d4bf",
            "graph": "v3_testing",
            "feeAmount":500,
            "LOW_BOLLINGER": 6,
            "MED_BOLLINGER": 4,
            "HIGH_BOLLINGER": 2,
            "PAST_WINDOW": 10,
            "requiredDay": "10-07-2021"
          }
        }
        
        Args::
            query: query based on v3 subgraph, e.g. poolDayDatas
            address: address of the Uniswap V3 pool
            graphMode: to differiante between testing and production graphs
            feeAmount: fee based on pool, e.g. 0.3% = 3000, 1% = 10000
            LOW_BOLLINGER: Bollinger multiplier for low risk profile, e.g. rangeValue = price +/- LOW_BOLLINGER*STD_DAYS
            MED_BOLLINGER: Bollinger multiplier for medium risk profile
            HIGH_BOLLINGER: Bollinger multiplier for high risk profile
            PAST_WINDOW: Past Days Window for Bollinger
            requiredDay: The day for which recommendation is needed, "dd-mm-yyyy" format
    '''

    response = {}
    transactionResponse = {}
    
    # Using Exception handling to catch errors in parsing the data
    try:
        event = json.loads(event) # NOTE: Comment out this line when deploying on AWS Lambda
        # Getting Event Body
        payload = event["body"]

        # Parsing the inputs
        queryType = payload["query"]
        poolAddress = payload["address"]
        graphMode = payload["graph"]
        feeAmount = payload["feeAmount"]
        tickSpacing = feeMapping[str(feeAmount)]
        requiredDay = payload["requiredDay"]
        LOW_BOLLINGER = int(payload["LOW_BOLLINGER"])
        MED_BOLLINGER = int(payload["MED_BOLLINGER"])
        HIGH_BOLLINGER = int(payload["HIGH_BOLLINGER"])
        PAST_WINDOW = int(payload["PAST_WINDOW"])

        # Other mode comparisons can be added here later
        if (graphMode == "v3_testing"):
            GRAPH_URL = GRAPH_URL_TESTING
    except Exception as e:

        '''
            Exception Handling formatted for lambda
        '''
        
        
        # If any error faced in parsing the above keys
        s = str(e)

        response["statusCode"] = 400
        transactionResponse["error"] = True
        transactionResponse["message"] = s 
        response["headers"] = {}
        response["headers"]["Content-Type"] = "application/json"

        response["body"] = transactionResponse

        # Converting python dictionary to JSON Object
        response_JSON = response
        
        # Revert the Response
        return response_JSON
    
    
    try:
        # Creating Query architecture

        query_transport=RequestsHTTPTransport(
        url=GRAPH_URL,
        verify=True,
        retries=5,
        )

        client = Client(
            transport=query_transport
        )


        datelt = dateToEpoch(requiredDay)

        # Creating the query string
        # Inserting pool addreess and date
        query_str_list = [
        "query {", 
        str(queryType)+ str(GRAPH_API_DICT[queryType][0][0])+str(poolAddress)+str(GRAPH_API_DICT[queryType][0][1])+str(datelt)+str(GRAPH_API_DICT[queryType][0][2])
        ]

        query_str_list_1 = [str(x) for x in GRAPH_API_DICT[queryType][1]]
        query_str_list.extend(query_str_list_1)
        query_str_list.extend(["}"])

        query_str = "\n".join(query_str_list)
#         print("Checking query string")
#         print(type(query_str))
#         print(query_str)

        # Creating the actual query
        query = gql(query_str)

        # Executing the query
        queryResponse = client.execute(query)

        # Creating dataframe for the respose
        responseDf = pd.DataFrame(queryResponse[queryType])

        # Formatting the date
        if (responseDf.columns[0] == "date"):
            responseDf['date'] = responseDf['date'].apply(DateTimeConverter)

        # Creating a transaction response to send back as response
        transactionResponse["graphData"] = {
            "date": list(responseDf["date"]),
            "sqrtPrice":list(responseDf["sqrtPrice"].astype(str)),
            "tick": list(responseDf["tick"].astype(str)),
            "price":list(pow(1.0001, responseDf["tick"].astype(float))),
            "token0Price": list(pd.to_numeric(responseDf["token0Price"])),
            "token1Price": list(pd.to_numeric(responseDf["token1Price"]))
        }


        
        # Check for NaN values in the transactionResponse graphData for price
        # Remove indexes that have null values for tick, usually happens for the first value
        idx_to_delete = []
        for i in range(len(transactionResponse["graphData"]["price"])):
            if (transactionResponse["graphData"]["price"][i] is None): 
                    idx_to_delete.append(i)
                    
        # If there is index to delete
        if idx_to_delete:
            # Removing the values which needs to be deleted for each key
            for key in transactionResponse["graphData"].keys():
                updated_values = []
                for i in range(len(transactionResponse["graphData"][key])):
                    if(i not in idx_to_delete):
                        updated_values.append(transactionResponse["graphData"][key][i])
                transactionResponse["graphData"][key] = updated_values


        # Final position recommendations
        transactionResponse["liquidityData"] = {
        "low":[],
        "medium":[],
        "high":[],
        }

        # Strategy creation

        # Moving Average
        mAvg = pd.Series(transactionResponse["graphData"]["token1Price"]).rolling(PAST_WINDOW).mean().iloc[-1]
        # STD
        std0 = pd.Series(transactionResponse["graphData"]["token1Price"]).rolling(PAST_WINDOW).std().iloc[-1]



        # Latest price, corresponding to the token1 price
        price0 = transactionResponse["graphData"]["token1Price"][-1]
        # Current tick value for price
        currentTick = tickcalc(price0)

        # Low-risk strategy
        llow = price0 - std0 * LOW_BOLLINGER
        lhigh = price0 + std0 * LOW_BOLLINGER
        
        # Negative price handling, just manually set to 2 width, can be modified according to need, no particular reason for it
        llow = price0 - (std0 * LOW_BOLLINGER)*2 if llow<=0 else llow

        # Calculating the tick values for lower and upper range to check if they are equal, 
        # especially for USDC/USDT type pools
        l_lowerTick = round(pricetickcalc(llow)/tickSpacing)*tickSpacing
        l_higherTick = round(pricetickcalc(lhigh)/tickSpacing)*tickSpacing

        # Edge case handling for same ticks
        if (l_lowerTick == l_higherTick):
            llow = priceFromTick(math.floor(currentTick/tickSpacing) * tickSpacing)
            lhigh = priceFromTick(math.ceil(currentTick/tickSpacing) * tickSpacing)



        # Adding position recommendation for low strategy, as (lower_range, price, upper_range)
        transactionResponse["liquidityData"]["low"].append(( str(max(llow, 0)), str(max(price0, 0)), str(max(lhigh, 0))))

        # Medium-risk strategy
        mlow = price0 - std0 * MED_BOLLINGER
        mhigh = price0 + std0 * MED_BOLLINGER
        
        mlow = price0 - (std0 * MED_BOLLINGER)*2 if mlow<=0 else mlow 

        m_lowerTick = round(pricetickcalc(mlow)/tickSpacing)*tickSpacing
        m_higherTick = round(pricetickcalc(mhigh)/tickSpacing)*tickSpacing

        # Edge case handling
        if (m_lowerTick == m_higherTick):
            mlow = priceFromTick(math.floor(currentTick/tickSpacing) * tickSpacing)
            mhigh = priceFromTick(math.ceil(currentTick/tickSpacing) * tickSpacing)


        # Adding position recommendation for medium strategy, as (lower_range, price, upper_range)
        transactionResponse["liquidityData"]["medium"].append(( str(max(mlow, 0)), str(max(price0, 0)), str(max(mhigh, 0))))

        # High-risk strategy
        hlow = price0 - std0 * HIGH_BOLLINGER
        hhigh = price0 + std0 * HIGH_BOLLINGER
        hlow = price0 - (std0 * MED_BOLLINGER)*2 if hlow<=0 else hlow 

        h_lowerTick = round(pricetickcalc(hlow)/tickSpacing)*tickSpacing
        h_higherTick = round(pricetickcalc(hhigh)/tickSpacing)*tickSpacing

        # Edge case handling
        if (h_lowerTick == h_higherTick):
            hlow = priceFromTick(math.floor(currentTick/tickSpacing) * tickSpacing)
            hhigh = priceFromTick(math.ceil(currentTick/tickSpacing) * tickSpacing)

        # Adding position recommendation for high strategy, as (lower_range, price, upper_range)
        transactionResponse["liquidityData"]["high"].append(( str(max(hlow, 0)), str(max(price0, 0)), str(max(hhigh, 0))))




    except Exception as e:
        # If any error faced in parsing the above keys
        s = str(e)

        response["statusCode"] = 400
        transactionResponse["error"] = True
        transactionResponse["message"] = s 
        response["headers"] = {}
        response["headers"]["Content-Type"] = "application/json"

        response["body"] = transactionResponse

        # Converting python dictionary to JSON Object
        response_JSON = response
        
        # Revert the Response
        return response_JSON




    response["statusCode"] = 200
    transactionResponse["error"] = False
    transactionResponse["message"] = "Error free execution"
    response["headers"] = {}
    response["headers"]["Content-Type"] = "application/json"

    response["body"] = transactionResponse

    # Converting python dictionary to JSON Object
    response_JSON = response

    return response



 
    

In [26]:
## Comment this part when using lambda
if __name__ == "__main__":

    input_data = {
          "body": {
            "query": "poolDayDatas",
            "address": "0x7858e59e0c01ea06df3af3d20ac7b0003275d4bf",
            "graph": "v3_testing",
            "feeAmount":500,
            "LOW_BOLLINGER": 6,
            "MED_BOLLINGER": 4,
            "HIGH_BOLLINGER": 2,
            "PAST_WINDOW": 10,
            "requiredDay": "10-07-2021"
          }
        }

    # Creating Input JSON
    input_json = json.dumps(input_data)
    print("Test JSON")
#     print(input_json)
    
    # Getting the result
    result = lambda_handler(input_json,"Context")
    
    # Checking the keys
    print(result["body"].keys())
    print()
    
    # Printing the liquidity recommendations
    print("Position Recommendations for differnt risk profiles")
    print(result["body"]["liquidityData"])
    

Test JSON
dict_keys(['graphData', 'liquidityData', 'error', 'message'])

Position Recommendations for differnt risk profiles
{'low': [('0.9996496044480653', '1.000202886055262', '1.0007561676624588')], 'medium': [('0.9998340316504642', '1.000202886055262', '1.00057174046006')], 'high': [('1.0', '1.000202886055262', '1.001000450120021')]}
